In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import time
from numba import jit
inicio = time.time()

In [2]:
def extract_data():
    raiz = Path.cwd() #capturo diretório atual
    pasta_dados = Path.joinpath(raiz,"dados") #defino onde estão os problemas
    prob_por_arquivo = 200 #informando quantos problemas por arquivo eu tenho
    conjuntos = [10,20,50,100,200,500,1000]

    for conjunto in conjuntos:  #circulo nos arquivos
        dados[conjunto] = {}
        arquivo = f"sch"+str(conjunto)+'.txt'
        print("Iniciando problema {}".format(arquivo))
        caminho_problema = Path.joinpath(pasta_dados,arquivo)
        df = pd.read_fwf(caminho_problema) #importo no df
        df.columns=["n","p","a","b"]
        df.drop(columns=["n"], inplace=True) #jogo fora columa inútil
        n=df.iloc[0,0] #guardo o tamanho de itens do problema
        print("n = ", n)  
        df.dropna(inplace=True) #jogo fora linhas inúteis
        for problema_numero in range(prob_por_arquivo): #vou circular nos problemas dentro do arquivo. Cada iteração deste for é um problema.
            prob = df[n*problema_numero:n*(problema_numero+1)]
            dados[conjunto][problema_numero+1] = {
                'pi': prob["p"].to_numpy(),
                'ai': prob["a"].to_numpy(),
                'bi': prob["b"].to_numpy()
            }
            ## nesse ponto tenho o parâmetros do problema atual guardadas nas variáveis pi, ai e bi, prontos para serem utilizados na modelagem.
    return dados

In [3]:
dados = {}
objetivos = {}
variaveis = {}
tempos = {}
dados = extract_data()
lista_hs = [0.8, 0.6, 0.4, 0.2]

h=0.8
conjunto = 1000
problema = 1
z_corte = 2

pi = []
ai = []
bi = []

pi = np.array(dados[conjunto][problema]['pi'])
ai = np.array(dados[conjunto][problema]['ai'])
bi = np.array(dados[conjunto][problema]['bi'])

d=sum(pi)*h

i_itens = len(pi)
i_index=list(range(i_itens))
i_np = np.array(i_index)

Iniciando problema sch10.txt
n =  10
Iniciando problema sch20.txt
n =  20
Iniciando problema sch50.txt
n =  50
Iniciando problema sch100.txt
n =  100
Iniciando problema sch200.txt
n =  200
Iniciando problema sch500.txt
n =  500
Iniciando problema sch1000.txt
n =  1000


In [4]:
sa = tuple([True,False]*int(i_itens*0.5)) #solucao atual

In [5]:
@jit(nopython=True)
def gera_lista_subZcorte(z_corte, zi):
    return np.nonzero(np.abs(zi) < z_corte)[0]

In [6]:
def transforma(solucao_partida):
    set_E = np.nonzero(solucao_partida == True)[0] #[0] para retornar o np array e não o tuple.
    set_T = np.nonzero(solucao_partida == False)[0]
    return set_E,set_T



@jit(nopython=True)
def calcula_objetivo(set_E,set_T):
    if np.sum(pi[set_E])>d:

        return 1e15 #retorna valor imenso.

    ai_pi = ai[set_E]/pi[set_E] #apenas do set_E
    bi_pi = bi[set_T]/pi[set_T] #apenas do set_T
    set_E_pi = pi[set_E]
    set_T_pi = pi[set_T]
    ai_pi_decr = np.flip(np.argsort(ai_pi)) #ordem de ai_pi e depois por pi.
    bi_pi_decr = np.flip(np.argsort(bi_pi))
    set_E = set_E[ai_pi_decr]
    set_T = set_T[bi_pi_decr]
    
    
    sum_pi = 0
    objetivo = 0
    for tarefa in set_E:
        objetivo += sum_pi * ai[tarefa]
        sum_pi += pi[tarefa]
    
    sum_pi = 0
    for tarefa in set_T:
        objetivo += (sum_pi+pi[tarefa])*bi[tarefa]
        sum_pi += pi[tarefa]


    
    return objetivo#,True
                           
@jit(nopython=True)
def busca_total_z(solucao_partida,z_corte): 
    objetivos = []
    lista_z = (bi-ai)/(bi+ai)
    servicos_abaixo_de_abs_zcorte = np.nonzero(np.abs(lista_z) < z_corte)[0]
    for cada_serv in servicos_abaixo_de_abs_zcorte:
        sol_em_teste = np.array(solucao_partida)
        sol_em_teste[cada_serv] = not(solucao_partida[cada_serv]) #inverte o serviço de bolso.
        set_E = np.nonzero(sol_em_teste == True)[0] #[0] para retornar o np array e não o tuple.
        set_T = np.nonzero(sol_em_teste == False)[0]
        objetivos.append(calcula_objetivo(set_E,set_T))
    return objetivos

In [7]:
%time objs = busca_total_z(sa , 0.5)

Wall time: 14.5 s


In [8]:

%time objs = busca_total_z(sa , 0.5)

Wall time: 43.9 ms


In [9]:
np.average(objs)

10428141.852201259

In [10]:
objs

[10439118.0,
 10417997.0,
 10430903.0,
 10406030.0,
 10442542.0,
 10425651.0,
 10436652.0,
 10432527.0,
 10409075.0,
 10430617.0,
 10417236.0,
 10431686.0,
 10425706.0,
 10432634.0,
 10411319.0,
 10431153.0,
 10440044.0,
 10437376.0,
 10425234.0,
 10428796.0,
 10419522.0,
 10438419.0,
 10430970.0,
 10432249.0,
 10444209.0,
 10414318.0,
 10434131.0,
 10414326.0,
 10430845.0,
 10419358.0,
 10455775.0,
 10426404.0,
 10428167.0,
 10447334.0,
 10427392.0,
 10439961.0,
 10429138.0,
 10430422.0,
 10437163.0,
 10408151.0,
 10441155.0,
 10457913.0,
 10427859.0,
 10446658.0,
 10411295.0,
 10422776.0,
 10420755.0,
 10416311.0,
 10438853.0,
 10440044.0,
 10426319.0,
 10424090.0,
 10430923.0,
 10413341.0,
 10421577.0,
 10432694.0,
 10418965.0,
 10434383.0,
 10441877.0,
 10429297.0,
 10428051.0,
 10428443.0,
 10423050.0,
 10424533.0,
 10441118.0,
 10439505.0,
 10422677.0,
 10432015.0,
 10429298.0,
 10432465.0,
 10406319.0,
 10422802.0,
 10437156.0,
 10429331.0,
 10426309.0,
 10429411.0,
 10426244.0,

In [11]:
lista_z

NameError: name 'lista_z' is not defined